# Bloodhound Playbook - Last Password Set
-----------------------------------------


## Import Libraries

In [12]:
from py2neo import Graph
import datetime as dt

## Initialize BloodHound neo4j Database Connection

In [13]:
g = Graph("bolt://127.0.0.1:7687", auth=("neo4j", "neo4j"))
g

<Graph database=<Database uri='bolt://127.0.0.1:7687' secure=False user_agent='py2neo/4.2.0 neobolt/1.7.15 Python/3.7.4-final-0 (win32)'> name='data'>

## Retrieve Enabled User Accounts With A Password Older than 10 years

Now we’ll construct the Cypher query that show us enabled user accounts with a password that is older than 10 years 

In [57]:
lastpwdset_df = g.run("""
MATCH (U:User) WHERE U.enabled = true AND U.pwdlastset < (datetime().epochseconds - (10 * 31556926)) 
AND NOT U.pwdlastset IN [-1.0, 0.0] 
RETURN U.name, U.pwdlastset
""").to_data_frame()

In [ ]:
# test = dt.datetime.utcfromtimestamp(1228117326).replace(tzinfo=datetime.timezone.utc)
lastpwdset_df

## Retrieve Users with SPN(Kerberoastable) And A Password Older Than 5 Years

In [45]:
lastpwdset_spn_df = g.run("""
MATCH (U:User) WHERE U.hasspn = true AND U.enabled = true AND U.pwdlastset < (datetime().epochseconds - (5 * 31556926)) 
AND NOT U.pwdlastset IN [-1.0, 0.0] 
RETURN U.name, U.pwdlastset, U.serviceprincipalnames
""").to_data_frame()

In [ ]:
lastpwdset_spn_df

## Which Of The Above Mentioned Users Have A Path To Domain Admin

In [38]:
lastpwdset_spn_da_df = g.run("""
MATCH (U:User) WHERE U.hasspn = true AND U.enabled = true AND U.pwdlastset < (datetime().epochseconds - (5 * 31556926)) 
AND NOT U.pwdlastset IN [-1.0, 0.0]
MATCH (g:Group {name:'DOMAIN ADMINS@TESTLAB.NET'})
MATCH p = shortestPath(
  (U)-[*1..]->(g)
)
RETURN U.name,LENGTH(p)
ORDER BY LENGTH(p) ASC
""").to_data_frame()

In [ ]:
lastpwdset_spn_da_df

## Local Admin Rights On Computers

Further, how many computers does the above mentioned users have local admins rights on.

In [41]:
comp_admin_df = g.run("""
MATCH (u:User) WHERE u.hasspn = true AND u.enabled = true AND u.pwdlastset < (datetime().epochseconds - (5 * 31556926)) 
AND NOT u.pwdlastset IN [-1.0, 0.0]
OPTIONAL MATCH (u)-[:AdminTo]->(c1:Computer)
OPTIONAL MATCH (u)-[:MemberOf*1..]->(:Group)-[:AdminTo]->(c2:Computer)
WITH u,COLLECT(c1) + COLLECT(c2) AS tempVar
UNWIND tempVar AS comps
RETURN u.name,COUNT(DISTINCT(comps))
ORDER BY COUNT(DISTINCT(comps)) DESC
""").to_data_frame()

In [ ]:
comp_admin_df

## Retrive Local Admin Rights For Observed Accounts

How many of the user accounts with old passwords have local admin rights?

In [51]:
local_admin_df = g.run("""
MATCH (u:User) WHERE u.enabled = true AND u.pwdlastset < (datetime().epochseconds - (10 * 31556926)) 
AND NOT u.pwdlastset IN [-1.0, 0.0] 
OPTIONAL MATCH (u)-[:AdminTo]->(c1:Computer)
OPTIONAL MATCH (u)-[:MemberOf*1..]->(:Group)-[:AdminTo]->(c2:Computer)
WITH u,COLLECT(c1) + COLLECT(c2) AS tempVar
UNWIND tempVar AS comps
RETURN u.name,COUNT(DISTINCT(comps))
ORDER BY COUNT(DISTINCT(comps)) DESC
""").to_data_frame()

In [ ]:
local_admin_df